# Accident Propensity Index Calculation v02
Using the original accident data. However, no efficiency measures taken yet (except that the df only has the ID and the start lat and lng).

In [7]:
import pandas as pd
import math
import time

# Define a function to calculate the distance between two points
def distance(point1, point2):
    lat1, lon1 = point1
    lat2, lon2 = point2
    km_per_lat = 110.574 # km per degree latitude
    km_per_lon = 111.320 # km per degree longitude at the equator
    dx = (lon2 - lon1) * km_per_lon * math.cos((lat1 + lat2) / 2)
    dy = (lat2 - lat1) * km_per_lat
    return math.sqrt(dx**2 + dy**2)

# Define a function to calculate the distance between a point and a line segment
def distance_to_segment(point, segment_start, segment_end):
    px, py = point
    x1, y1 = segment_start
    x2, y2 = segment_end
    dx, dy = x2 - x1, y2 - y1
    segment_length_squared = dx*dx + dy*dy
    if segment_length_squared == 0:
        return distance(point, segment_start)
    t = max(0, min(1, ((px - x1) * dx + (py - y1) * dy) / segment_length_squared))
    x = x1 + t * dx
    y = y1 + t * dy
    return distance(point, (x, y))

# Define a function to find accidents on a given route within a maximum distance
def find_accidents_on_route(start_point, end_point, max_distance):
    # Create a mask for accidents that are within the maximum distance from the route
    mask = data.apply(lambda row: distance_to_segment((row['Start_Lat'], row['Start_Lng']), start_point, end_point) <= max_distance, axis=1)

    # Return the accidents that match the mask
    return data.loc[mask]

In [9]:
# Accident data frame
data = pd.read_csv('accident_locations.csv')

# Maximal distance of accidents from route in kilometers
max_distance = 0.05

In [19]:
# Route start and end point
start_point = (33.77299, -84.39020)
end_point = (33.777495, -84.390903)

In [20]:
# Run after entering the accident, route, and distance data
start_time = time.time()
accidents = find_accidents_on_route(start_point, end_point, max_distance)
print("Accidents on Route")
print(accidents)
print("\nAccidents off Route")
not_on_route = ~data['ID'].isin(accidents['ID'])
print(data[not_on_route])
end_time = time.time()
print("Time taken: {:.2f} seconds".format(end_time-start_time))

Accidents on Route
                ID  Start_Lat  Start_Lng
32295      A-32296   33.77758  -84.39076
32461      A-32462   33.77758  -84.39076
33763      A-33764   33.77758  -84.39076
34198      A-34199   33.77758  -84.39076
37982      A-37983   33.77758  -84.39076
...            ...        ...        ...
2797884  A-2797885   33.77758  -84.39076
2809620  A-2809621   33.77758  -84.39076
2810470  A-2810471   33.77758  -84.39076
2835549  A-2835550   33.77758  -84.39076
2840352  A-2840353   33.77758  -84.39076

[252 rows x 3 columns]

Accidents off Route
                ID  Start_Lat   Start_Lng
0              A-1  40.108910  -83.092860
1              A-2  39.865420  -84.062800
2              A-3  39.102660  -84.524680
3              A-4  41.062130  -81.537840
4              A-5  39.172393  -84.492792
...            ...        ...         ...
2845337  A-2845338  34.002480 -117.379360
2845338  A-2845339  32.766960 -117.148060
2845339  A-2845340  33.775450 -117.847790
2845340  A-2845341  33.9

In [22]:
import folium

# randomly select x entries from the data DataFrame
# sample_data = data.sample(n=1000, random_state=42)
sample_data = accidents

# create a map object centered at the mean latitude and longitude of the accidents
map_accidents = folium.Map(location=[sample_data.Start_Lat.mean(), sample_data.Start_Lng.mean()], zoom_start=10)

# add markers for each accident to the map
for index, row in sample_data.iterrows():
    folium.Marker(location=[row['Start_Lat'], row['Start_Lng']],
                  popup=f"Accident ID: {row['ID']}",
                 ).add_to(map_accidents)

# add start point marker to the map
folium.Marker(location=[start_point[0], start_point[1]],
              icon=folium.Icon(color='green', icon='glyphicon-home'),
              popup='Start Point',
             ).add_to(map_accidents)

# add end point marker to the map
folium.Marker(location=[end_point[0], end_point[1]],
              icon=folium.Icon(color='red', icon='glyphicon-flag'),
              popup='End Point',
             ).add_to(map_accidents)

# display the map
map_accidents